In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import sqlite3
import pandas as pd

In [3]:
# create sqlite connection and cursor
# and attached needed databases, created if they do not exist
Con = sqlite3.connect('')
Cur = Con.cursor()
Cur.execute("ATTACH DATABASE 'hedgefund.db' AS hedgefund")
Cur.execute("ATTACH DATABASE 'rfasset.db' AS rfasset")
pd.DataFrame(Cur.execute('pragma database_list').fetchall()) #list db

,0,1,2
0,0,main,
1,2,hedgefund,/Users/emilyziyixiao/DataspellProjects/AdvDsFi...
2,3,rfasset,/Users/emilyziyixiao/DataspellProjects/AdvDsFi...


[a]

In [6]:
# Load the data from the Excel file into pandas dataframes
hedgefund_df = pd.read_excel('data.xlsx', sheet_name='HedgeFund', header=1, index_col=None)
hedgefund_df = hedgefund_df[:-3]
hedgefund_df = hedgefund_df.rename(columns={hedgefund_df.columns[0]: "Year"})
hedgefund_df["Year"] = hedgefund_df["Year"].astype(int)
hedgefund_df['YTD'] = hedgefund_df['YTD'].astype(str).str.strip('%').astype(float) / 100

# Connect to the databases and create the tables
conn_hedge = sqlite3.connect("hedgefund.db")
hedgefund_df.to_sql("hf_wide", conn_hedge, if_exists="replace", index=False)

# Query the data to make sure it was stored correctly
check_table_hf_wide = pd.read_sql_query("SELECT * FROM hf_wide", conn_hedge)
check_table_hf_wide.head()

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,YTD
0,1997,0.0362,0.0118,-0.0099,0.0049,0.0406,0.0311,0.0470,0.0052,0.0405,-0.0127,-0.0019,0.0129,0.2235
1,1998,-0.0019,0.0376,0.0354,0.0115,-0.0168,0.0023,-0.0046,-0.0781,0.0105,0.0217,0.0378,0.0295,0.0823
2,1999,0.0223,-0.0084,0.0348,0.0492,0.0115,0.0413,0.0082,0.0032,0.0074,0.0190,0.0533,0.0773,0.3656
3,2000,0.0095,0.0676,0.0174,-0.0147,-0.0100,0.0326,0.0006,0.0374,-0.0071,-0.0107,-0.0252,0.0220,0.1219
4,2001,0.0304,-0.0144,-0.0111,0.0202,0.0147,0.0051,-0.0057,-0.0018,-0.0258,0.0184,0.0202,0.0171,0.0677


[b]

In [7]:
# Load the data from the Excel file into pandas dataframes
rfasset_df = pd.read_excel('data.xlsx', sheet_name='RiskFreeAsset',header=1, index_col=None)
rfasset_df['RF'] = rfasset_df['RF'] / 100

# Connect to the databases and create the tables
conn_rf = sqlite3.connect("rfasset.db")
rfasset_df.to_sql('rf_long', conn_rf, if_exists='replace', index=False)

# Query the data to make sure it was stored correctly
check_table_rf_long = pd.read_sql_query("SELECT * FROM rf_long", conn_rf)
check_table_rf_long.head()

,Date,RF
0,192607,0.0022
1,192608,0.0025
2,192609,0.0023
3,192610,0.0032
4,192611,0.0031
